# KONUT FİYATLARININ TAHMİNİ

Konut fiyat tahmini oldukça karmaşık ve çok sayıda parametrenin dikkate alınmasını gerektiren bir tahmin işlemidir.

Veri kümesinde 1460 konutun 81 adet kolon bulunmaktadır. Bu kolonların 79 tanesi konut özelliklerini temsil etmektedir. Konut özelliklerini temsil eden 79 kolonun 43 tanesi kategorik, 36 tanesi sayısal değer almaktadır. Konut özelliklerini temsil eden kolonlarda eksik veriler bulunmaktadır.

Bu çalışmanın amacı bu özellikler dikkate alınarak konut satış fiyatının(SalePrice) tahmin edilmesidir.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ycimpute

İlk adım olarak projede kullanacağımız Python’a ait olan kütüphanelerini import edelim.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ycimpute.imputer import knnimput, MICE
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

Pandas kütüphanesini import ederken pd kısaltması ile tanımladığımız için kodun her alanında artık pd ile çağırabilmemiz mümkündür. Üzerinde çalışacağım veri kümesi bir CSV dosyası olduğu için read_csv( ) metodu ile verilerimizi okuyalım.

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

info( ) metodu ile çalışılan veri kümesi içerisindeki dtype olarak bilinen veri tipleri ve kullanılan bellek hakkında bize bilgi vermektedir. Böylece veri kümesinin kolon sayısını ve veri tiplerini öğrenebiliriz.

In [ ]:
train.info()

# SATIŞ FİYATININ İNCELENMESİ

Bu çalışmadaki asıl önemli nokta konutların satış fiyatı yani "SalePrice" kolonudur. Satış fiyatının ve diğer konut özelliklerinin birbiri ile olan korelasyonlarını yani birbirleriyle olan bağlılıklarını incelemek bize özellik seçiminde yardımcı olacaktır.

"SalePrice" kolonunu biraz inceleyelim.

In [ ]:
train_salePrice = train["SalePrice"]
train_salePrice

In [ ]:
#konut fiyatlarının istatiksel özeti

salePrice_desc = train['SalePrice'].describe().T
salePrice_desc 

In [ ]:
sns.distplot(train['SalePrice'])
plt.ylabel('Frekans')
plt.title('SalePrice Dağılımı')

skew() metodu veri kümesinin çarpıklığını hesaplamaktadır. Çarpıklık; 

![](http://media.geeksforgeeks.org/wp-content/uploads/skewness.jpg)

formulü ile hesaplanır. 

* çarpıklık = 0: normal dağılım
* çarpıklık > 0: dağılımın sol kuyruğunda daha fazla ağırlık
* çarpıklık < 0: dağılımın sağ kuyruğunda daha fazla ağırlık 

Basıklık kavramı bir reel değerli rassal değişken için olasılık dağılımının, grafik gösteriminden tanımlanarak ortaya çıkarılan bir kavram olan, sivriliği veya basıklığı özelliğinin ölçümüdür.



In [ ]:
#Çarpıklık ve basıklık

print("Çarpıklık: %f" % train['SalePrice'].skew())
print("Basıklık: %f" % train['SalePrice'].kurt())

corr() metodu veri kümesinde yer alan özeliklerin korelasyon matrisini oluşturur(Sadece sayılsal değer alan özellikler için). Korelasyon iki rassal değişken arasındaki doğrusal ilişkinin yönünü ve gücünü belirtir.

In [ ]:
# korelasyon matrisi

corrmat = train.corr()
corrmat

In [ ]:
# korelasyon matrisinin görselleştirilmesi

plt.subplots(figsize=(15,12))
sns.heatmap(corrmat, vmax=1, cmap="Blues", square=True)

Bu çalışmada önemli olan nokta konutların satış fiyatı(SalePrice) olduğu için bir de satış fiyatının diğer özelliklerle olan korelasyonuna bakalım.

In [ ]:
#"SalePrice" ve diğer konut özelliklerinin korelasyon matrisinin oluşturulması

salePrice = train["SalePrice"]
corr_salePrice = train.corrwith(salePrice, axis=0)
corr_salePrice = pd.DataFrame(corr_salePrice)
corr_salePrice.rename (columns = {0: 'SalePrice'}, inplace = True)

#Oluşturulan korelasyon matrisinin görselleştirilmesi
plt.subplots(figsize=(15,12))
sns.set(font_scale=1.1)
sns.heatmap(corr_salePrice, vmax=1, cmap="Blues",fmt='.4f',annot=True);

Yukarıda yer alan korelasyon grafiğinde satış fiyatının ve diğer konut özelliklerinin korelasyonu görülmektedir. Buradan yola çıkarak bir konutun satış fiyatını 0.5'ten büyük bir korelasyonu olan özelliklerin ‘OverallQual’, 'YearBuilt', ‘TotalBsmtSF’, ‘1stFlrSF’, 'GrLiArea', 'FullBath','TotRmsAbvGrd', ‘GarageCars’ ve ‘GarageArea’ olduğu görülmektedir.

Bu özelliklerin korelasyonuna bakacak olursak;

In [ ]:
k = 10 # heatmap için değişken sayısı
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values, cmap="Blues")
plt.show()

Şimdi gelin konut fiyatlarının belirlenmesinde önemli olan bu özellikleri biraz yakından inceleyelim.

OverallQual kolonu konutun malzeme kalitesini ifade etmektedir. 1-10 arasında sayısal değer almaktadır.

In [ ]:
#box plot overallqual/saleprice
data = pd.concat([train['SalePrice'], train['OverallQual']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data);
fig.axis(ymin=0, ymax=800000);

Kutu grafikleri, araştırmacıların ortalama değerleri, veri kümesinin dağılımını ve değişkenlik belirtilerini hızlı bir şekilde tanımlamasını, verilerin görsel bir özeti sağlayan grafiklerdir. Yukarıdaki grafikte aykırı değerler var gibi görünse de bu akırılık konutun sahip olduğu diğer özellikler ile bağlantılıdır.

Bir diğer önemli özelliğimiz de 'YearBuilt' yani konutun yapım yılı. Bu özelliğin kutu grafiği ise şu şekilde;

In [ ]:
# YearBuild ve SalePrice boxlot
plt.figure(figsize=(40,20))
sns.set(font_scale=1.5)
sns.boxplot(x='YearBuilt', y="SalePrice", data=train);
sns.swarmplot(x='YearBuilt', y="SalePrice", data=train, color=".25");
plt.xticks(weight='bold',rotation=90);

Dağılım (scatter) diyagramı, iki farklı değişkenin arasındaki ilişkiyi belirlemek için kullanılan grafiklerden biridir. 

Satış fiyatı ve giriş katının toplam metrekaresi(TotalBsmtSF) arasındaki ilişki şu şekildedir;

In [ ]:
# TotalBsmtSF/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['TotalBsmtSF']], axis=1)
data.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0,800000));

Satış fiyatı ve konutun 1. katının metrekasi(1stFlrSF) arasındaki ilişki;

In [ ]:
#1stFlrSF/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['1stFlrSF']], axis=1)
data.plot.scatter(x='1stFlrSF', y='SalePrice', ylim=(0,800000));

Satış fiyatı ve konutun zemin üzerindeki toplam yaşam alanı(GrLivArea) arasındaki ilişki;

In [ ]:
#GrLivArea/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['GrLivArea']], axis=1)
data.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0,800000));

Satış fiyatı ile giriş katının toplam metrekaresi(TotalBsmtSF), konutun 1. katının metrekasi(1stFlrSF) ve konutun zemin üzerindeki toplam yaşam alanı(GrLivArea) arasında doğrusal bir ilişki bulunduğu dağılım grafiklerinde görülmektedir.


Satış fiyatı ve konutun tamamlanmış banyo sayısı (FullBath) arasındaki ilişki;

In [ ]:
#FullBath/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['FullBath']], axis=1)
data.plot.scatter(x='FullBath', y='SalePrice', ylim=(0,800000));

Satış fiyatı ve konutta yer alan toplam oda sayısı (TotRmsAbvGrd) arasındaki ilişki;

In [ ]:
#TotRmsAbvGrd/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['TotRmsAbvGrd']], axis=1)
data.plot.scatter(x='TotRmsAbvGrd', y='SalePrice', ylim=(0,800000));

Satış fiyatı ve konutun garajının araç kapasitesi(GarageCars) arasındaki ilişki;

In [ ]:
#GarageCars/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['GarageCars']], axis=1)
data.plot.scatter(x='GarageCars', y='SalePrice', ylim=(0,800000));

Satış fiyatı ve konutun garajının metrekaresi (GarageArea) arasındaki ilişki;

In [ ]:
#GarageArea/salePrice Dağılım Grafiği 

data = pd.concat([train['SalePrice'], train['GarageArea']], axis=1)
data.plot.scatter(x='GarageArea', y='SalePrice', ylim=(0,800000));

In [ ]:
train = train.drop("SalePrice", axis='columns')
train.shape

In [ ]:
all_data = pd.concat([train,test])
all_data.head()

In [ ]:
all_data = all_data.reset_index(drop=True)

# EKSİK VERİLERİN İNCELENMESİ

Eksik veri incelenen veri kümesindeki gözlemlerde eksiklik olması durumunu ifade etmektedir. Veri ön işleme işlemlerinde karşımıza çıkan ciddi problemlerden biridir.

Öncelikle konut özelliklerini temsil eden kolonların hangisinde, ne kadar eksik veri olduğunu inceleyelim. Toplamda her sütun (özellik) için ne kadar NaN değer yer aldığını görmek istiyor iseniz sum() metodu ile sonucu ekranda görebilmeniz mümkündür.

In [ ]:
total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Toplam', 'Yüzde'])
missing_data = missing_data
missing_data.head(34)
missing_data

In [ ]:
# Eksik verilerin görselleştirilmesi

f,ax = plt.subplots(figsize=(15, 10))
missing = round(all_data.isnull().mean(),5)
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color="b")

ax.set(ylabel="Eksik değerlerin yüzdesi")
ax.set(xlabel="Özellikler")
ax.set(title= "Özelliğe göre eksik veri yüzdesi")
sns.despine(trim=True, left=True)

Veri kümesinin özellikleri incelendiği zaman, veri kümesinde yer alan kategorik değerler alan özelliklerde bulunan NaN değerler aslında eksik veri olmadığı ve bir anlam ifade ettiği,'NA' yani yok anlamına geldiği görülmektedir.

Öncelikle veri kümesinde bu duruma sahip olan özelliklerideki eksik değerleri ‘NA’ ifadesi ile tamamlamaya başlayalım.

In [ ]:
missing_categorical_cols = ["BsmtFinType2","BsmtQual","BsmtCond","BsmtExposure",
                            "BsmtFinType2","GarageCond", "GarageQual", "GarageFinish",
                            "GarageType","FireplaceQu","Fence",
                            "Alley", "MiscFeature", "PoolQC"]

for col in missing_categorical_cols:
    col = all_data[col]
    for index in range(col.size):
        if col[index] is np.nan:
           col[index]= "NA"
        else:
            continue

Veri kümelerindeki bazı özelliklerdeki eksiklikler başka bir değişkenle bağlantılı olup yapısal bir durumdan dolayı eksik olabilmektedir.

Konutun garajının yapım yılı(GarageYrBlt) özelliğindeki eksik veriler de yapısal durumdan dolayı ortaya çıkmıştır. Bu özellik ‘NA’ ifadesi ile doldurulan eksik verilerde bağlantılı olup garajı olmayan konutların garaj yapım yılı da bulunmamaktadır.

In [ ]:
garageYrBlt = all_data["GarageYrBlt"]
for index in range(garageYrBlt.size):
    if np.isnan(garageYrBlt[index]):
        garageYrBlt[index] = 0.0
    else:
        continue

Eksik verinin 10 taneden az olduğu özellikleri Scikit-learn kütüphanesinin SimpleImputer modülünü kullanarak en sık kullanılan değer ile tamamlayalım.

Scikit-learn, Python programlama dilinde yazılmış bir makine öğrenme kütüphanesidir. SimpleImputer eksik değerlerin dönüştürülmesini sağlayan sklearn kütüphanesinden kullandığımız bir dönüştürücü modüldür.

missing_values parametresi ile dönüştürülmek istenen veriler belirlenmektedir.Daha sonraki parametre olan strategy parametresi sırasında ise 4 adet yer alan strateji türünden birisini seçerek hangi yönteme göre dönüştürüleceğine karar verilmektedir. Aşağıda bu stratejilere yer verilmektedir:

“mean” stratejisi, her sütun boyunca ortalama kullanılarak eksik değerlerin değiştirilmesini sağlar. Yalnızca sayısal verilerle kullanılabilir.

“median” stratejisi, her sütun boyunca medyan kullanılarak eksik değerlerin değiştirilmesini sağlar. Yalnızca sayısal verilerle kullanılabilir.

“most_frequent” stratejisi , her sütun boyunca en sık kullanılan değeri kullanılarak eksik değerlerin değiştirilmesini sağlar. String veya sayısal verilerle kullanılabilir.

“constant” stratejisi, eksik değerleri fill_value ile değiştirin. String veya sayısal verilerle kullanılabilir.

In [ ]:
mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtFinType1'] = mean_f.fit_transform(all_data[['BsmtFinType1']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['Electrical'] = mean_f.fit_transform(all_data[['Electrical']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['MSZoning'] = mean_f.fit_transform(all_data[['MSZoning']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['Functional'] = mean_f.fit_transform(all_data[['Functional']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['Utilities'] = mean_f.fit_transform(all_data[['Utilities']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtHalfBath'] = mean_f.fit_transform(all_data[['BsmtHalfBath']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtFullBath'] = mean_f.fit_transform(all_data[['BsmtFullBath']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['KitchenQual'] = mean_f.fit_transform(all_data[['KitchenQual']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['Exterior2nd'] = mean_f.fit_transform(all_data[['Exterior2nd']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['Exterior1st'] = mean_f.fit_transform(all_data[['Exterior1st']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtFinSF1'] = mean_f.fit_transform(all_data[['BsmtFinSF1']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['SaleType'] = mean_f.fit_transform(all_data[['SaleType']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['TotalBsmtSF'] = mean_f.fit_transform(all_data[['TotalBsmtSF']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtUnfSF'] = mean_f.fit_transform(all_data[['BsmtUnfSF']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['BsmtFinSF2'] = mean_f.fit_transform(all_data[['BsmtFinSF2']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['GarageCars'] = mean_f.fit_transform(all_data[['GarageCars']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['GarageArea'] = mean_f.fit_transform(all_data[['GarageArea']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['MasVnrType'] = mean_f.fit_transform(all_data[['MasVnrType']])

mean_f = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
all_data['MasVnrArea'] = mean_f.fit_transform(all_data[['MasVnrArea']])


In [ ]:
masVnrType = all_data['MasVnrType']

for index in range(masVnrType.size):
    if masVnrType[index] == 'None':
        masVnrType[index] = 'NA'
    else:
        continue

all_data['MasVnrType'] = masVnrType

all_data['MasVnrType']

In [ ]:
total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Toplam', 'Yüzde'])

missing_data[:20]

Yapılan eksik veri tamamlama işlemlerinden sonra sadece konuta bağlı caddenin uzunluğu (LotFrontage) özelliğinde eksik veriler bulunmaktadır. Bu özellikteki eksik verilerin sayısı fazla olduğu için tahmine dayalı değer atama yöntemi kullanalım.

Değer atama işlemi için uygulaması kolay gözetimli öğrenme algoritmalarından K-En Yakın Komşu (KNN) algoritmasından ve çoklu değer atama (multiple imputation) yöntemlerinden faydalanacağız ancak bundan önce kategorik özelliklerimizi sayısal özelliklere dönüştürmemiz gerekmekte. Bunun için get_dummies() metodunu kullanacağız.

In [ ]:
centralAir = train["CentralAir"]

for index in range(centralAir.size):
    if centralAir[index] is "N":
        centralAir[index] = 0
    else:
        centralAir[index] = 1

    
train["CentralAir"] = centralAir

Konutun bünyesinde barındırdığı bazı özelliklerin kalitesi ile ilgili özellikler kategorik değişken olarak yer almaktadır. Öncelikle bu özellikleri sayısal değerlere dönüştürelim.

In [ ]:
alley = all_data['Alley']
HasAlley = []
AlleyGrvl = []
AlleyPave = []

for index in range(alley.size):
    if alley[index] == 'NA':
        HasAlley.append(0)
    else:
        HasAlley.append(1)   
    if alley[index] == "Grvl":
        AlleyGrvl.append(1)
    else:
        AlleyGrvl.append(0)   
    if alley[index] == "Pave":
        AlleyPave.append(1)
    else:
        AlleyPave.append(0)

all_data["HasAlley"] = HasAlley
all_data["AlleyGrvl"] = AlleyGrvl
all_data["AlleyPave"] = AlleyPave

all_data = all_data.drop("Alley", axis='columns')
all_data.head()

In [ ]:
BsmtQual_ = all_data["BsmtQual"]
BsmtQual = []
HasBsmt = []

for index in range(BsmtQual_.size):
    if BsmtQual_[index] == "NA":
        BsmtQual.append(0)
        HasBsmt.append(0)
    else:
        HasBsmt.append(1)
    if BsmtQual_[index] == "Ex":
        BsmtQual.append(105)   
    if BsmtQual_[index] == "Gd":
        BsmtQual.append(95)   
    if BsmtQual_[index] == "TA":
        BsmtQual.append(85)     
    if BsmtQual_[index] == "Fa":
        BsmtQual.append(75)
    if BsmtQual_[index] == "Po":
        BsmtQual.append(65)

all_data["HasBsmt"] = HasBsmt
all_data["BsmtQual"] = BsmtQual

In [ ]:
BsmtCond_ = all_data["BsmtCond"]
BsmtCond = []

for index in range(BsmtCond_.size):
    if BsmtCond_[index] == "NA":
        BsmtCond.append(0)
    if BsmtCond_[index] == "Ex":
        BsmtCond.append(5)     
    if BsmtCond_[index] == "Gd":
        BsmtCond.append(4)    
    if BsmtCond_[index] == "TA":
        BsmtCond.append(3)    
    if BsmtCond_[index] == "Fa":
        BsmtCond.append(2)
    if BsmtCond_[index] == "Po":
        BsmtCond.append(1)

all_data["BsmtCond"] = BsmtCond

In [ ]:
GarageQual_ = all_data["GarageQual"]
HasGarage = []
GarageQual = []

for index in range(GarageQual_.size):
    if GarageQual_[index] == "NA":
        HasGarage.append(0)
        GarageQual.append(0)
    else:
        HasGarage.append(1) 
    if GarageQual_[index] == "Ex":
        GarageQual.append(5)     
    if GarageQual_[index] == "Gd":
        GarageQual.append(4)     
    if GarageQual_[index] == "TA":
        GarageQual.append(3)    
    if GarageQual_[index] == "Fa":
        GarageQual.append(2)
    if GarageQual_[index] == "Po":
        GarageQual.append(1)

all_data["GarageQual"] = GarageQual
all_data["HasGarage"] = HasGarage

In [ ]:
GarageCond_ = all_data["GarageCond"]
GarageCond = []

for index in range(GarageCond_.size):
    if GarageCond_[index] == "NA":
        GarageCond.append(0)
    if GarageCond_[index] == "Ex":
        GarageCond.append(5)  
    if GarageCond_[index] == "Gd":
        GarageCond.append(4)     
    if GarageCond_[index] == "TA":
        GarageCond.append(3)     
    if GarageCond_[index] == "Fa":
        GarageCond.append(2)
    if GarageCond_[index] == "Po":
        GarageCond.append(1)

all_data["GarageCond"] = GarageCond

In [ ]:
PoolQC_ = all_data["PoolQC"]
HasPool = []
PoolQC = []


for index in range(PoolQC_.size):
    if PoolQC_[index] == "NA":
        HasPool.append(0)
        PoolQC.append(0)
    else:
        HasPool.append(1)
    if PoolQC_[index] == "Ex":
        PoolQC.append(4)    
    if PoolQC_[index] == "Gd":
        PoolQC.append(3)    
    if PoolQC_[index] == "TA":
        PoolQC.append(2)     
    if PoolQC_[index] == "Fa":
        PoolQC.append(1)

all_data["PoolQC"] = PoolQC
all_data["HasPool"] = HasPool

In [ ]:
FireplaceQu_ = all_data["FireplaceQu"]
HasFireplace = []
FireplaceQu = []

for index in range(FireplaceQu_.size):
    if FireplaceQu_[index] == "NA":
        HasFireplace.append(0)
        FireplaceQu.append(0)
    else:
        HasFireplace.append(1)
    if FireplaceQu_[index] == "Ex":
        FireplaceQu.append(5)     
    if FireplaceQu_[index] == "Gd":
        FireplaceQu.append(4)        
    if FireplaceQu_[index] == "TA":
        FireplaceQu.append(3)       
    if FireplaceQu_[index] == "Fa":
        FireplaceQu.append(2)   
    if FireplaceQu_[index] == "Po":
        FireplaceQu.append(1)

all_data["FireplaceQu"] = FireplaceQu
all_data["HasFireplace"] = HasFireplace

In [ ]:
Fence_ = all_data["Fence"]
HasFence = []
Fence = []

for index in range(Fence_.size):
    if Fence_[index] == "NA":
        HasFence.append(0)
        Fence.append(0)
    else:
        HasFence.append(1)
    if Fence_[index] == "GdPrv":
        Fence.append(4)      
    if Fence_[index] == "MnPrv":
        Fence.append(3)   
    if Fence_[index] == "GdWo":
        Fence.append(2)
    if Fence_[index] == "MnWw":
        Fence.append(1)

all_data["Fence"] = Fence
all_data["HasFence"] = HasFence

In [ ]:
ExterQual_ = all_data["ExterQual"]
ExterQual = []

for index in range(ExterQual_.size):
    if ExterQual_[index] == "NA":    
        ExterQual.append(0)
    if ExterQual_[index] == "Ex":
        ExterQual.append(5)   
    if ExterQual_[index] == "Gd":
        ExterQual.append(4)    
    if ExterQual_[index] == "TA":
        ExterQual.append(3)
    if ExterQual_[index] == "Fa":
        ExterQual.append(1)
    if ExterQual_[index] == "Po":
        ExterQual.append(0)

all_data["ExterQual"] = ExterQual

In [ ]:
ExterCond_ = all_data["ExterCond"]
ExterCond = []

for index in range(ExterCond_.size):
    if ExterCond_[index] == "NA":    
        ExterCond.append(0)
    if ExterCond_[index] == "Ex":
        ExterCond.append(5)  
    if ExterCond_[index] == "Gd":
        ExterCond.append(4) 
    if ExterCond_[index] == "TA":
        ExterCond.append(3)
    if ExterCond_[index] == "Fa":
        ExterCond.append(2)
    if ExterCond_[index] == "Po":
        ExterCond.append(1)

all_data["ExterCond"] = ExterCond

In [ ]:
HeatingQC_ = all_data["HeatingQC"]
HeatingQC = []

for index in range(HeatingQC_.size):
    if HeatingQC_[index] == "NA": 
        HeatingQC.append(0)  
    if HeatingQC_[index] == "Ex":
        HeatingQC.append(5)      
    if HeatingQC_[index] == "Gd":
        HeatingQC.append(4)     
    if HeatingQC_[index] == "TA":
        HeatingQC.append(3)
    if HeatingQC_[index] == "Fa":
        HeatingQC.append(2)
    if HeatingQC_[index] == "Po":
        HeatingQC.append(1)

all_data["ExterCond"] = ExterCond

In [ ]:
KitchenQual_ = all_data["KitchenQual"]
KitchenQual = []

for index in range(KitchenQual_.size):
    if KitchenQual_[index] == "NA": 
        KitchenQual.append(0)
    if KitchenQual_[index] == "Ex":
        KitchenQual.append(5)    
    if KitchenQual_[index] == "Gd":
        KitchenQual.append(4)  
    if KitchenQual_[index] == "TA":
        KitchenQual.append(3)
    if KitchenQual_[index] == "Fa":
        KitchenQual.append(2)
    if KitchenQual_[index] == "Po":
        KitchenQual.append(1)

all_data["KitchenQual"] = KitchenQual


In [ ]:
all_data.head()

In [ ]:
#all_data'nın dummy edilmesi.
all_data_copy = all_data.copy()
all_data_copy = all_data.drop("Id", axis='columns')

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorial_cols = []

for col in all_data_copy.columns:
    if all_data_copy[col].dtype not in numeric_dtypes:
        categorial_cols.append(col)
    else:
        continue

all_data_dummy = pd.get_dummies(all_data_copy[categorial_cols])
all_data_copy = all_data.drop(categorial_cols, axis='columns')
all_data_dummy  = pd.concat([all_data_copy,all_data_dummy], axis=1)

all_data_dummy.head()

In [ ]:
all_data_dummy.info()

Veri kümemizdeki kategorik özelliklerimizi sayısal özelliklere dönüştürdük. Şimdi kalan eksik verilerimizi de tamamlayalım. Bunun için Ycimpute kütüphanesinin Knnimpute modülünü kullanacağım. Ama bu modülü kullanabilmek için önce install edip sonra import etmemiz gerekmekte.

knnimput() metodu bizden bir Numpy array'i beklediği için veri kümemizdeki özellikleri bir yerde saklayıp veri kümemizi bir Numpy array'e dönüştürüp eksik değerleri tamamladıktan sonra tekrar isimlendirme işleminin gerçekleştireceğiz;

In [ ]:
#LotFrontage kolonunun KNN ile doldurulması
var_names = list(all_data_dummy)
array_all_data = np.array(all_data_dummy)
all_data_knn_dummy = knnimput.KNN(k = 4).complete(array_all_data)
all_data_knn_dummy = pd.DataFrame(all_data_knn_dummy, columns = var_names)

In [ ]:
total = all_data_knn_dummy.isnull().sum().sort_values(ascending=False)
percent = (all_data_knn_dummy.isnull().sum()/all_data_knn_dummy.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Toplam', 'Yüzde'])

missing_data

In [ ]:
all_data_knn_dummy.head(20)

In [ ]:
all_data_knn = all_data.copy()
all_data_knn["LotFrontage"] = all_data_knn_dummy["LotFrontage"]
all_data_knn.head(20)

Çoklu değer atama yönteminde mevcut veri kümesi üzerinden bir istatistiksel dağılım elde edilir (örneğin ilkelleme (regression)) ardından bu dağılım sayesinde eksik verileri dolduran bir bağlantı kullanılır, bu işlem birden fazla yapılarak her bir veri kümesi daha sonra kullanılmak üzere saklanır.

In [ ]:
#Multiple Imputation(MICE) ile tamamlanması
var_names = list(all_data_dummy)
array_all_data = np.array(all_data_dummy)
all_data_mice_dummy = MICE().complete(array_all_data)
all_data_mice_dummy = pd.DataFrame(all_data_mice_dummy, columns = var_names)
all_data_mice_dummy.head(20)

In [ ]:
all_data_mice = all_data.copy()
all_data_mice["LotFrontage"] = all_data_mice_dummy["LotFrontage"]
all_data_mice.head(20)

# ÖZELLİK SEÇİMİ

Çok fazla özellik kullanmak modelin performansını düşürebilir. En iyi performans sonucu elde etmek için, en optimum değişkenler belirlenip bu özellikler ile model kurmak gerekmektedir. Seçilen özellikler ile karmaşık olmayan daha kolay anlaşılır  ve hızlı çalışan bir model elde edilmiş olunur.

**1. VARYANSA GÖRE ÖZELLİK SEÇİMİ**

Varyansa göre özellik seçimi düşük varyanslı özelliklerin veri kümesinden çıkarılmasıdır. Varyans, verilerin aritmetik ortalamadan sapmalarının karelerinin toplamıdır. Eğer bütün değerler aynı ise varyans = 0 olacaktır. Bu aşama tüm değerleri aynı olan kategorik özelliklerin veri kümesinden çıkarma işleminin sayısal özellikler için olan aşamasıdır. 

In [ ]:
#Varyansa göre özellik seçimi

from sklearn.feature_selection import VarianceThreshold


selector = VarianceThreshold()
selected_knn_dummy_vt = selector.fit_transform(all_data_knn_dummy)
selected_knn_dummy_vt = pd.DataFrame(selected_knn_dummy_vt)
selected_knn_dummy_vt

Varyansa göre özellik seçimi yapıldığında alt küme olarak bütün özelliklerin yer aldığı bir veri kümesi elde edilmiştir, bu durumda veri kümesindeki özelliklerde yüksek oranda aynı değeri taşıyan bir özellik bulunmadığını göstermektedir.


**2. KORELASYONA GÖRE ÖZELLİK SEÇİMİ**

Veri kümesindeki özelliklerin satış fiyatıyla doğrusal ilişkinin yönünü ve gücününü  Pearson   korelasyonu ile gözlemlemiştik. Pearson korelasyonu sayısal değişkenler için kullanılmaktadır. Pearson korelasyonu ile özellik seçimi yapmak da mümkündür. 

In [ ]:
#korelasyonlara göre özellik seçimi

train_knn = all_data_knn_dummy.copy()
train_knn = train_knn[:1460]
train_knn["SalePrice"] = train_salePrice

cor_knn = train_knn.corr()

#Hedef değişken ile olan korelasyon
cor_target_knn = abs(cor_knn["SalePrice"])

selected_knn_corr = cor_target_knn[cor_target_knn > 0.5]

selected_knn_corr

In [ ]:
train_mice = all_data_mice_dummy.copy()
train_mice = train_mice[:1460]
train_mice["SalePrice"] = train_salePrice

cor_mice = train_mice.corr()#korelasonu yönü önemli olmadığı şiddeti bizim için önemli o yüzden mutlak değerini alıyoruz.
cor_target_mice = abs(cor_mice["SalePrice"])

selected_mice_corr = cor_target_mice[cor_target_mice > 0.5]#Yüksek korelasyonlu olanları seçme

selected_mice_corr

**3. AĞAÇ TABANLI ÖZELLİK SEÇİMİ**

Ekstra ağaç sınıflandırıcısı, sınıflandırma sonucunu çıkarmak için bir "ormanda" toplanan birden fazla ilişkisiz karar ağacının sonuçlarını bir araya getiren bir toplu öğrenme tekniğidir. Her ağacın tahmini dikkate alınarak nihai kararın alındığı çok sayıda karar ağacından oluşmaktadır

In [ ]:
#Agac tabanlı özellik seçimi
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

salePrice = []
for i in range(1459):
    salePrice.append(0)
    
X_knn = all_data_knn_dummy
y = list(train_salePrice) + salePrice
y = pd.Series(y)

clf = ExtraTreesClassifier(n_estimators=10)
clf = clf.fit(X_knn, y)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
selected_knn_etc = model.transform(X_knn)
selected_knn_etc = pd.DataFrame(selected_knn_etc)
selected_knn_etc

In [ ]:

import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LogisticRegression

X_SFS = all_data_knn_dummy
y = list(train_salePrice) + salePrice
y = pd.Series(y)

# Sequential Forward Selection(sfs)
sfs = SFS(LogisticRegression(),
           k_features=30,
           forward=True,
           floating=False,
           scoring = 'accuracy',
           cv = 0)
sfs1=sfs.fit(X_SFS, y)
fig1 = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
result_LR = pd.DataFrame.from_dict(sfs1.get_metric_dict(confidence_interval=0.90)).T
result_LR.sort_values('avg_score', ascending=0, inplace=True)
result_LR.head()

best_features_LR = result_LR.feature_idx.head(1).tolist()
select_features_LR = can.columns[best_features_LR]
select_features_LR

In [ ]:
X_train = all_data_knn_dummy[:1460].drop("Id", axis='columns')
y_train = train_salePrice
X_test = all_data_knn_dummy[1460:].drop("Id", axis='columns')


rf_model = RandomForestRegressor(n_estimators=10, max_depth=20)
tuned = rf_model.fit(X_train,y_train)
y_pred = tuned.predict(X_test)

In [ ]:
my_submission = pd.DataFrame({'Id': range(1461,2920), 'SalePrice': y_pred })
my_submission.to_csv('submission_rf.csv', index=False)
my_submission.info()

In [ ]:
print(x)

# **MODEL OLUŞTURULMASI**

Örneklem üzerinden elde edilen tahmin fonksiyonu;

Y' =b0 + b1x1 + b2x2 +...+bjxj+...+ pxp   
Y' : Konut fiyatı
bi  : Konutun fiyatını etkileyen özelliklerin ağırlıkları
xi  : Konut fiyatını etkileyen özellikler


Probleme, veri kümesine göre değişiklik gösteren makine öğrenmesi modellerinin parametrelerini hiper parametre (hyperparameters) olarak adlandırılmaktadır.  Bu hiper parametreler probleme,  veri kümesine göre farklılık gösterdiği için hiper parametre seçimi modeli oluşturan tasarımcı tarafından yapılmaktadır.

Scikit-learn kütüphanesinde yer alan GridSearchCV modülü de hiper parametre seçiminde kullanılmaktadır.
GridSearchCV, modelde denenmesi istenen hiper parametreler ve değerleri için bütün kombinasyonlar ile ayrı ayrı model kurarak ve belirtilen metriğe göre en başarılı hiper parametre seti belirlemektedir.

**1. RASTGELE ORMAN ALGORİTMASI (RANDOM FOREST)**

Topluluk öğrenme yöntemi olan rastgele orman algoritması, sınıflandırma işlemi esnasında birden fazla karar ağacı üreterek sınıflandırma değerini yükseltmeyi hedefleyen bir algoritmadır.


In [ ]:
X_train = all_data_knn_dummy[:1460].drop("Id", axis='columns')
y_train = train_salePrice
X_test = all_data_knn_dummy[1460:].drop("Id", axis='columns')


rf_model = RandomForestRegressor()

rf_params = { "max_depth" : [2,4,5,8,10],
              "max_features" : [1,2,5,8,10,17],
              "n_estimators" : [100,200,500,1000,1500,1800,2000],
              "min_samples_split": [2,10,75,60,100,80] }

rf_cv_model = GridSearchCV(rf_model, rf_params, cv=10, n_jobs=-1, verbose=2).fit(X_train,y_train)

rf_best_params = rf_cv_model.best_params_

md = rf_best_params["max_depth"]
mf = rf_best_params["max_features"]
mss = rf_best_params["min_samples_split"]
n = rf_best_params["n_estimators"]

rf_model = RandomForestRegressor(max_depth = md, max_features = mf, min_samples_split = mss, n_estimators = n)
rf_tuned = rf_model.fit(X_train,y_train)

y_pred = rf_tuned.predict(X_test)
y_pred 


In [ ]:
rf_Importance = pd.DataFrame({"Importance":rf_tuned.feature_importances_*100},index = X_train.columns)

s = rf_Importance.sort_values(by = "Importance", axis=0, ascending = False)
fig_dims = (50, 50)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "Importance", y =  rf_Importance["Importance"].index, ax=ax, data=s);



In [ ]:
new_train_columns = []

for index, row in rf_Importance.iterrows():
    if(row["Importance"] >= 0.1):
        new_train_columns.append(index)
    
new_train = X_train[new_train_columns]

X_train = new_train
X_test = X_test[new_train_columns]
rf_model = RandomForestRegressor(max_depth = md, max_features = mf, min_samples_split = mss, n_estimators = n)
rf_tuned = rf_model.fit(X_train,y_train)

y_pred = rf_tuned.predict(X_test)

 **2. XGBOOST ALGORİTMASI**

XGBOOST algoritması da ağaca dayalı makine öğrenme algoritmalarından biridir. Gradient Boosting algoritmasının çeşitli düzenlemeler ile optimize edilmiş yüksek performanslı halidir. 

In [ ]:
# MICE ile eksik verilerin tamamlanana veri kümesinin eğitilmesi 

X_train = all_data_mice_dummy[:1460].drop("Id", axis='columns')
y_train = train_salePrice
X_test = all_data_mice_dummy[1460:].drop("Id", axis='columns')

In [ ]:
# KNN yöntemi ile eksik verilerin tamamlanana veri kümesinin eğitilmesi 

X_train = all_data_knn_dummy[:1460].drop("Id", axis='columns')
y_train = train_salePrice
X_test = all_data_knn_dummy[1460:].drop("Id", axis='columns')

In [ ]:
# Korelasyon tabanlı özellik seçimi ile oluşturulan veri kümesinin eğitilmesi 

train_columns = ["OverallQual","YearBuilt","YearRemodAdd", "ExterQual", "TotalBsmtSF" ,    
                 "1stFlrSF", "GrLivArea", "FullBath", "KitchenQual", "TotRmsAbvGrd","GarageCars", "GarageArea"]

X_train = all_data_knn_dummy[:1460].drop("Id", axis='columns')
X_train = X_train[train_columns]
y_train = train_salePrice
X_test = all_data_knn_dummy[1460:].drop("Id", axis='columns')
X_test = X_test[train_columns]

In [ ]:
# Ağaç tabanlı özellik seçimi ile oluşturulan veri kümesinin eğitilmesi 

X_train = selected_knn_etc[:1460]
y_train = train_salePrice
X_test = selected_knn_etc[1460:]

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

xgb_model = XGBRegressor()

xgb_params = { "learning_rate" : [0.1,0.01,0.001],
               "max_depth" : [3, 5, 8, 10],
               "n_estimators" : [100,200,500,1000,1500,1800,2000],
               "colsample_bytree": [0.4, 0.7, 1] }

xgb_cv_model = GridSearchCV(xgb_model, xgb_params, cv=10, n_jobs=-1, verbose=2).fit(X_train,y_train)


xgb_model = xgb_cv_model.best_estimator_
xgb_tuned = xgb_model.fit(X_train,y_train)

y_pred_xgb = xgb_tuned.predict(X_test)  

In [ ]:
xgb_model

In [ ]:
xgb_Importance = pd.DataFrame({"Importance":xgb_tuned.feature_importances_*100},index = X_train.columns)

s = xgb_Importance.sort_values(by = "Importance", axis=0, ascending = False)
fig_dims = (50, 50)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "Importance", y =  xgb_Importance["Importance"].index, ax=ax, data=s)


In [ ]:
new_train_columns = []

for index, row in xgb_Importance.iterrows():
    if(row["Importance"] >= 0.1):
        new_train_columns.append(index)
    
new_train = X_train[new_train_columns]

X_train = new_train
X_test = X_test[new_train_columns]

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

xgb_model = xgb_cv_model.best_estimator_
xgb_tuned = xgb_model.fit(X_train,y_train)

y_pred_xgb = xgb_tuned.predict(X_test)
y_pred_xgb

In [ ]:
my_submission = pd.DataFrame({'Id': range(1461,2920), 'SalePrice': y_pred_xgb })
my_submission.to_csv('submission_xgb.csv', index=False)
my_submission.info()

**ÇOK KATMANLI YAPAY SİNİR AĞLARI**

İnsan beyninin bilgi işleme şeklini referans alan sınıflandırma ve regresyon problemleri için kullanılabilen kuvvetli makine  öğrenmesi algoritmalarından biridir.

In [ ]:
# KNN ile eksik verileri tamamlanarak ağaç tabanlı özellik seçimi ile oluşturulan veri kümesinin eğitilmesi 

X_train = selected_knn_etc[:1460].drop(0, axis='columns')
y_train = train_salePrice
X_test = selected_knn_etc[1460:].drop(0, axis='columns')

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

mlp_model = MLPRegressor()

mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001],
               "hidden_layer_sizes": [(50,50), (10,10), (150,100),(20,10), (250,250), (300,100),(100,100)],
               "solver" : ["lbfgs","adam","sgd"],
               "activation": ["relu","logistic","tanh"]}

mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv=10, n_jobs=-1, verbose=2).fit(X_train_scaled,y_train)

mlp_model = mlp_cv_model.best_estimator_
mlp_model

mlp_tuned = mlp_model.fit(X_train_scaled,y_train)

y_pred_mlp = mlp_tuned.predict(X_test_scaled)
y_pred_mlp


In [ ]:
# KNN ile eksik verileri tamamlanan bütün veri kümesinin eğitilmesi 

X_train = all_data_knn_dummy[:1460].drop("Id", axis='columns')
y_train = train_salePrice
X_test = all_data_knn_dummy[1460:].drop("Id", axis='columns')


scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)


mlp_model = MLPRegressor()

mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001],
               "hidden_layer_sizes": [(50,50), (10,10), (150,100),(200,100), (250,250), (300,100),(100,100)],
               "solver" : ["lbfgs","adam","sgd"],
               "activation": ["relu","logistic","tanh"]}

mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv=10, n_jobs=-1, verbose=2).fit(X_train_scaled,y_train)

mlp_model = mlp_cv_model.best_estimator_
mlp_tuned = mlp_model.fit(X_train_scaled,y_train)

y_pred_mlp = mlp_tuned.predict(X_test_scaled)
y_pred_mlp

In [ ]:
my_submission = pd.DataFrame({'Id': range(1461,2920), 'SalePrice': y_pred_mlp })
my_submission.to_csv('submission_mlp.csv', index=False)
my_submission.info()

In [ ]:
print("done")